In [0]:
# Create widgets for Silver layer
dbutils.widgets.text("source_table", "default.bronze_events")

source_table = dbutils.widgets.get("source_table")

print("Running Silver layer")
print("Source Table:", source_table)

In [0]:
from pyspark.sql import functions as F

silver_df = (
    spark.read.table(source_table)
    .filter(F.col("price") > 0)
    .filter(F.col("price") < 10000)
    .dropDuplicates(["user_session", "event_time"])
    .withColumn("event_date", F.to_date("event_time"))
    .withColumn(
        "price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "nid")
         .otherwise("premium")
    )
)

silver_df = silver_df.drop("ingestion_ts")

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.silver_events")

print("Silver layer completed successfully")